In [1]:
print("Enter keyword you'd like to crawl")
keyword = input()

Enter keyword you'd like to crawl
대구 명소


In [2]:
import requests
from urllib.request import Request
import urllib.parse as parse
import urllib.request as request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains as AC
import json
import pandas as pd
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re

In [55]:
def getGoogleReviews(keyword, driver):
    googleReviews = pd.DataFrame(columns=['name', 'date', 'reviews', 'stars', 'category', 'address', 'lat', 'lng', 'foreigner'])
    url = ('https://www.google.co.kr/maps/search/'+keyword+
               '/@35.8698753,128.5880298,15z/data=!3m1!4b1?hl=ko')
    driver.get(url)
    while True:
        googleReviews = pd.concat([googleReviews, getReviewsOnThisPage(driver)])
        time.sleep(3)
        try: 
            nextPage = driver.find_element_by_xpath("//button[@aria-label='다음 페이지']")
            nextPage.click()
            time.sleep(4)
        except:
            break
    return googleReviews

def getReviewsOnThisPage(driver):
    pageReviews = pd.DataFrame(columns=['name', 'date', 'reviews', 'stars', 'category', 'address', 'lat', 'lng', 'foreigner'])
    
    scrollPane = driver.find_element_by_xpath("//div[@class='section-layout section-scrollbox scrollable-y scrollable-show section-layout-flex-vertical']")
    for i in range(0, 5):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollPane)
        time.sleep(0.5)
        
    buttons = driver.find_elements_by_xpath("//div[@jsan='7.sJKr7qpXOXd__content-container,7.sJKr7qpXOXd__result-container-clickable,7.sJKr7qpXOXd__content-container-is-link,0.aria-label']")
    print(len(buttons))
    buttonList = list()
    for button in buttons:
        buttonList.append(button.get_attribute('innerHTML'))
        
    for i in range(0, len(buttonList)):
        soup = BeautifulSoup(buttonList[i],'html.parser')
        name = soup.find('div', class_='tYZdQJV9xeh__title gm2-subtitle-alt-1').text
        category = soup.find('div', class_='tYZdQJV9xeh__info-line').text
        button = driver.find_elements_by_xpath("//a[@jsan='7.place-result-container-place-link,0.aria-label,8.href,0.jsaction']")[i]
        button.click()
        time.sleep(2.5)
        review = visitReview(i, driver, name, category)
        pageReviews = pd.concat([pageReviews, review])
        

    return pageReviews

def visitReview(index, driver, title, category):
    #Getting inside the Review page
    reviewCnt = 0 
    #address
    address = ""
    #latitude and longtitude info
    #data=!4m8!1m2!2m1!1z64yA6rWs66qF7IaM!3m4!1s0x356609d9ab274835:0x879f109ab47152ee!8m2!3d35.8314654!4d128.6875637?hl=ko
    try: 
#         button.click()
#         time.sleep(2.5)
        address = driver.find_element_by_xpath("//button[@data-item-id='address']").get_attribute('aria-label')[4:]
        urlSplit = driver.current_url.split('!')
#         print(urlSplit[-2]+ ','+urlSplit[-1])
        latitude = urlSplit[-2][2:]
        longtitude = urlSplit[-1].split('?')[0][2:]
        button = driver.find_element_by_xpath("//button[@jsaction='pane.rating.moreReviews']")
        reviewCnt = int(button.get_attribute('aria-label')[3:-1].replace(",", ""))
        print(title,': ', reviewCnt)
        button.click()
        time.sleep(1)
    except:
        print('except: visitReview.getting into the page')
        url_before = driver.current_url
        while True:
            backButton = driver.find_element_by_xpath("//button[@jsaction='pane.place.backToList']")
            backButton.click()
            time.sleep(2)
            if url_before != driver.current_url:
                break
        return None
    
    #Scroll down for certain times
    scrollPane = driver.find_element_by_xpath("//div[@class='section-layout section-scrollbox scrollable-y scrollable-show']")
#     scrollTimes = (reviewCnt>1500) and 150 or reviewCnt // 10
#     scrollTimes = (reviewCnt>10) and 1 or reviewCnt // 10
    for i in range(0, reviewCnt//10):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollPane)
        time.sleep(0.5)
    
    stars = driver.find_elements_by_xpath("//span[@class='section-review-stars']")
    reviews = driver.find_elements_by_xpath("//span[@class='section-review-text']")
    dates = driver.find_elements_by_xpath("//span[@class='section-review-publish-date']")
    col_stars = list()
    col_reviews = list()
    col_dates = list()
    col_foreigner = list()
    
    hangul = re.compile('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]+')
    for i in range(0, len(stars)):
        review = reviews[i].text
        star = stars[i].get_attribute('aria-label')[4]
        date = getExactDateTime(dates[i].text)
        foreigner = 1 if (hangul.sub('', review).replace(" ", "")=='' or 
                         '(Google 번역 제공)' in review) else 0
        col_stars.append(star)
        col_reviews.append(review)
        col_dates.append(date)
        col_foreigner.append(foreigner)
    itemReviews = pd.DataFrame()
    itemReviews['reviews'] = col_reviews
    itemReviews['stars'] = col_stars
    itemReviews['name'] = title
    itemReviews['category'] = category
    itemReviews['address'] = address
    itemReviews['lat'] = latitude
    itemReviews['lng'] = longtitude
    itemReviews['date'] = col_dates
    itemReviews['foreigner'] = col_foreigner
#     print(address)

    itemReviews = itemReviews[['name', 'date', 'reviews', 'stars', 'category', 'address', 'lat', 'lng', 'foreigner']]
    #Getting out to the original list page
    try: 
        url_before = driver.current_url
        while True:
            #windows
#             backButton = driver.find_element_by_xpath("//button[@aria-label='뒤로']")
            #mac
            backButton = driver.find_element_by_xpath("//button[@aria-label='Back']")

#             print('success')
            backButton.click()
            time.sleep(1.5)
            if url_before != driver.current_url:
                break
        backButton = driver.find_element_by_xpath("//button[@jsaction='pane.place.backToList']")
        backButton.click()
        time.sleep(1)
#         url_before = driver.current_url
#         while True:
#             backButton = driver.find_element_by_xpath("//button[@jsaction='pane.place.backToList']")
#             backButton.click()
# #             driver.execute_script('window.history.go(-1)')
#             time.sleep(1)
#             if url_before != driver.current_url:
#                 break
    except:
        print('except: visitReview.getting out of the page')
        return None
    return itemReviews

def getExactDateTime(relativeDate):
    now = datetime.now()
    digit = int(re.findall('\d+', relativeDate)[0])
    count = relativeDate[-3]
    if count=='초':
        now = now + timedelta(seconds=-1*digit)
    elif count=='분':
        now = now + timedelta(minutes=-1*digit)
    elif count=='간':
        now = now + timedelta(hours=-1*digit)
    elif count=='일':
        now = now + timedelta(days=-1*digit)
    elif count=='주':
        now = now + timedelta(weeks=-1*digit)
    elif count=='달':
        now = now + relativedelta(months=-1*digit)
    elif count=='년':
        now = now + relativedelta(years=-100)
    return now.strftime("%Y/%m/%d")
#     time2 + timedelta(days=1, hours=-2)

In [56]:
# def getGoogleReviewsOnCertainPage(pageNum, keyword, driver):
#     googleReviews = pd.DataFrame(columns=['name', 'date', 'reviews', 'stars', 'category', 'address', 'lat', 'lng'])
#     url = ('https://www.google.co.kr/maps/search/'+keyword+
#                '/@35.8698753,128.5880298,15z/data=!3m1!4b1?hl=ko')
#     driver.get(url)
#     print('reading page', (pageNum+1), '...')
#     try:
#         for i in range(0, pageNum):
#             beforeButton = driver.find_element_by_css_selector("a[data-result-index]").get_attribute('aria-label')
#             while True:
#                 nextPage = driver.find_element_by_xpath("//button[@aria-label='다음 페이지']")
# #                 print('다음 페이지...')
#                 nextPage.click()
#                 time.sleep(2)
#                 afterButton = driver.find_element_by_css_selector("a[data-result-index]").get_attribute('aria-label')
#                 if beforeButton != afterButton:
#                     break
#     except:
#         return googleReviews

#     return getReviewsOnThisPage(driver)

In [57]:
# Ways to read all the data in one time
driver = webdriver.Chrome()
driver.implicitly_wait(10)
df = getGoogleReviews(keyword, driver)
driver.close()

6
 김광석 다시 그리기 길  :  6455
 아양기찻길  :  923


KeyboardInterrupt: 

In [ ]:
print('before drop: ', df.shape)
df_drop = df.drop_duplicates()
print('after drop: ', df_drop.shape)

In [ ]:
df_drop = df_drop.astype(str).apply(lambda x: x.str.encode('cp949', 'ignore').str.decode('cp949'))
df_drop.to_csv(keyword+'.csv', index=False, encoding='cp949')
# pd.read_csv(keyword+str(1)+'.csv', sep=',', encoding='cp949')

In [ ]:
# driver = webdriver.Chrome()
# driver.implicitly_wait(2)
# index = 1
# while True:
#     df = getGoogleReviewsOnCertainPage(index-1, keyword, driver)
#     if df.empty:
#         break
#     df = df.astype(str).apply(lambda x: x.str.encode('cp949', 'ignore').str.decode('cp949'))
#     df.to_csv(keyword+str(index)+'.csv', index=False, encoding='cp949')
#     index = index+1

reading page 1 ...
해랑정더덕밥 :  109


In [13]:
# df = pd.DataFrame(columns=['name', 'date', 'reviews', 'stars', 'category', 'address', 'lat', 'lng'])
# for i in range(1, index):
#     df = pd.concat([df, pd.read_csv(keyword+str(i)+'.csv', sep=',', encoding='cp949')])
# df.to_csv(keyword + '.csv', index=False, encoding='cp949')
# # pd.read_csv(keyword+str(1)+'.csv', sep=',', encoding='cp949')

In [14]:
# df = pd.read_csv(keyword+'.csv', sep=',', encoding='cp949')
# len(df.Name.unique())

24